# Gold層テーブル作成②: 顧客セグメント分析テーブル

このノートブックでは、Silver層から顧客セグメント別の購買行動分析用のGold層テーブルを作成します。

## 目的
- **顧客セグメンテーション**: 年齢層・性別・会員種別による分析
- **購買行動分析**: セグメント別の支出パターン把握
- **マーケティング戦略**: セグメント特性に基づく施策立案

## 作成テーブル
- **customer_segment_spending**: 顧客セグメント別支出分析

## セグメント分類
- **年齢層**: Under 30, 30-49, 50+
- **性別**: Male, Female
- **会員種別**: Premium, Regular, Basic

## 分析指標
- **avg_spend**: セグメント別平均支出額
- **num_transactions**: セグメント別取引回数


In [0]:
# Gold スキーマ設定
catalog_name = "users"
schema = "yukiteru_koide"

# スキーマ切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema}")


In [0]:
# Silver テーブル読み込み
df = spark.table(f"{catalog_name}.{schema}.transactions_enriched")

from pyspark.sql.functions import avg, count, when, col

# 年齢層を追加
df = df.withColumn("age_group", when(col("age") < 30, "Under 30")
                               .when(col("age") < 50, "30-49")
                               .otherwise("50+"))

segment_sales = (
    df
    .groupBy("age_group", "gender", "membership")
    .agg(
        avg("total_price").alias("avg_spend"),
        count("transaction_id").alias("num_transactions")
    )
)

segment_sales.write.format("delta").mode("overwrite").saveAsTable("customer_segment_spending")
